In [ ]:
import os

assert len(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) > 0, \
            "Set GOOGLE_APPLICATION_CREDENTIALS prior to use, see README.md"
assert len(os.environ['PYTHONPATH']) > 0, \
            "Set PYTHONPATH to include this repository prior to use, see README.md"

In [15]:
import pandas as pd
mini_traces_path = '/media/ponbac/GigaHDD/ethereum/mini-traces.csv'
traces_iter = pd.read_csv(mini_traces_path, chunksize=1000000, index_col=0)
for traces in traces_iter:
    traces: pd.DataFrame = traces
    for trace in traces.itertuples():
        print(trace[3])
        if len(str(trace[3])) < 10:
            print(f'from: {trace.from_address}, to: {trace.to_address}')

nan


/tmp/ipykernel_114738/2340755654.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for traces in traces_iter:


TypeError: object of type 'float' has no len()

In [ ]:
from utils.storage.base import EthereumStorage
from utils.storage.google_cloud_storage import GoogleCloudStorage

storage: EthereumStorage = GoogleCloudStorage()
storage.download('eth-aml-data', 'mini-traces.csv', './data/')
storage.download('eth-aml-data', 'sorted/blocks-sorted.csv', './data/')
storage.download('eth-aml-data', 'known-addresses.csv', './data/')
storage.download('eth-aml-data', 'tornado.csv', './data/')

In [ ]:
import pandas as pd
from utils.loader import DataLoader, DataframeLoader

BLOCK_CSV = f'./data/blocks-sorted.csv'
KNOWN_ADDRESSES_CSV = f'./data/known-addresses.csv'
CACHE_DIR = f'./data/cache'
TRACES_CSV = f'./data/mini-traces.csv'
TORNADO_CSV = f'./data/tornado.csv'
loader: DataLoader = DataframeLoader(block_csv=BLOCK_CSV, known_addresses_csv=KNOWN_ADDRESSES_CSV,
                                      cache_dir=CACHE_DIR, traces_csv=TRACES_CSV, tornado_csv=TORNADO_CSV)

In [ ]:
from src.blacklist.poison_blacklister import Poison
from src.blacklist.haircut_blacklister import Haircut
from src.blacklist.seniority_blacklister import Seniority
from src.blacklist.advanced_fifo_blacklister import AdvancedFIFO

os.makedirs('./results', exist_ok=True)

blacklist: pd.Series = loader.get_flagged().address
#algo = Poison(loader, blacklist, '/media/ponbac/BigHDD/ethereum/blacklist/testing/')
#algo = Haircut(loader, blacklist, '/media/ponbac/BigHDD/ethereum/blacklist/testing/')
#algo = Seniority(loader, blacklist, '/media/ponbac/BigHDD/ethereum/blacklist/testing/')
algo = AdvancedFIFO(loader, blacklist, './results/')
algo.make_blacklist(end_block=2000000)